In [42]:
from sqlalchemy import create_engine, text, inspect, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from flask import Flask, jsonify, render_template, request
from sqlalchemy.orm import sessionmaker
import json
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import pickle
from flask_cors import CORS
import numpy as np

import csv

In [43]:
app = Flask(__name__)

CORS(app)

In [44]:
#   Future_Enrollment_Data = Base.classes.Future_Enrollment
#     Enrollment_Data = Base.classes.Enrollment_2

#     # Iniciate Session
#     Session = sessionmaker(bind=engine)
#     session = Session()

#     Future_Enrollment_results = session.query(Future_Enrollment_Data).all()
#     Enrollment_results = session.query(Enrollment_Data).all()

#     # Extracting the data in the Enrollment table
#     Enrollment__data = [{column.name: str(getattr(result, column.name)) for column in Enrollment_Data.__table__.columns} for result in Enrollment_results]

#     # Extracting the data in the Future_Enrollment table
#     Future__Enrollment__data = [{column.name: str(getattr(result, column.name)) for column in Future_Enrollment_Data.__table__.columns} for result in Future_Enrollment_results]
    
#     # Closing Session 
#     session.close()

#     # Trasform the data in to Dataframes
#     Enrollment_df = pd.DataFrame(Enrollment__data)
#     Future_Enrollment_df = pd.DataFrame(Future__Enrollment__data)


In [52]:
Future_grad_df = pd.read_csv("future_data.csv")

In [54]:
Future_grad_df.head()

,Board Number,Year,Board Type,Region,Grade 10 OSSLT Results,Progress in Grade 10 OSSLT Results,Credit Accumulation by the end of Grade 10,Progress in Credit Accumulation by the end of Grade 10,Credit Accumulation by the end of Grade 11,Progress in Credit Accumulation by the end of Grade 11,Four Year Graduation Rate 2017-2018 Grade 9 Cohort,Progress in Four Year Graduation Rate 2017-2018 Grade 9 Cohort
0,B28010,2023,Public,North Region,0.72,0.0,0.69,-0.02,0.71,-0.08,0.709,-0.037
1,B28010,2024,Public,North Region,0.72,0.0,0.69,-0.02,0.71,-0.08,0.709,-0.037
2,B28010,2025,Public,North Region,0.72,0.0,0.69,-0.02,0.71,-0.08,0.709,-0.037
3,B28010,2026,Public,North Region,0.72,0.0,0.69,-0.02,0.71,-0.08,0.709,-0.037
4,B67202,2023,Roman Catholic,East Region,0.86,0.0,0.84,0.00,0.87,-0.03,0.882,-0.014


In [55]:
Graduation_df = pd.read_csv("graduation_data.csv")

In [56]:
Graduation_df.head()

,Board Number,Year,Board Type,Region,Grade 10 OSSLT Results,Progress in Grade 10 OSSLT Results,Credit Accumulation by the end of Grade 10,Progress in Credit Accumulation by the end of Grade 10,Credit Accumulation by the end of Grade 11,Progress in Credit Accumulation by the end of Grade 11,Four Year Graduation Rate 2017-2018 Grade 9 Cohort,Progress in Four Year Graduation Rate 2017-2018 Grade 9 Cohort
0,B28010,2021-2022,Public,North Region,0.72,0.0,0.69,-0.02,0.71,-0.08,0.718,0.018
1,B67202,2021-2022,Roman Catholic,East Region,0.86,0.0,0.84,0.00,0.87,-0.03,0.880,0.004
2,B66010,2021-2022,Public,West Region,0.81,0.0,0.81,0.03,0.78,-0.01,0.810,0.066
3,B66001,2021-2022,Public,West Region,0.75,0.0,0.67,-0.01,0.71,-0.01,0.711,0.011
4,B67164,2021-2022,Roman Catholic,West Region,0.85,0.0,0.73,-0.08,0.81,-0.01,0.797,-0.038


In [66]:
def predict(self, X):
    model_path = "graduation_prediction_model.pkl"

    # Importing the ML model
with open(model_path, "rb") as f:
    model = pickle.load(f)

        # Making predictions on the Future_Enrollment_df
    predictions = model.predict(Future_grad_df)

        # Importing predictions in the Future_Enrollment_df
    Future_grad_df["Four Year Graduation Rate 2017-2018 Grade 9 Cohort"] = predictions

    # Extracting the first year for each row and transforming to int
    Graduation_df["Year"] = Graduation_df["Year"].astype(str).str.split("-").str[0]    

    # Merge selected columns to Future_Enrollment_df based on School_Number
    Future_grad_df = Future_grad_df.merge(Graduation_df[["Board Number", "Board Type", "Region"]], on="Board Number", how="left")

    Complete_df = pd.concat([Graduation_df,Future_grad_df])

    # Selecting the data for a specific year
    #Final_df = Complete_df[Complete_df["Year"] == x]

    Final_df = Final_df.drop_duplicates(subset=["Board Number", "Year"])

    # Convert DataFrame back to list of dictionaries
    Complete__Graduations = Final_df.to_dict(orient='records')
    
    # Return Jsonify Board_data
return jsonify(Complete_df)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Board Number
- Board Type
- Four Year Graduation Rate 2017-2018 Grade 9 Cohort
- Region 


In [65]:
if __name__ == "__main__":
    app.run(debug = True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1